In [ ]:
##Import libraries 

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings("ignore")
from datetime import datetime, timedelta

# import all libraries and dependencies for data visualization
pd.options.display.float_format='{:.4f}'.format
plt.rcParams['figure.figsize'] = [8,8]
pd.set_option('display.max_columns', 350)
pd.set_option('display.max_colwidth', -1) 
pd.set_option("display.max_rows", 500)
sns.set(style='darkgrid')
import matplotlib.ticker as ticker
import matplotlib.ticker as plticker


# import all libraries and dependencies for machine learning
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
from sklearn.base import TransformerMixin
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import OrdinalEncoder
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import RobustScaler 
from sklearn.preprocessing import StandardScaler

from sklearn.compose import make_column_transformer


import statsmodels.api as sm
from sklearn.feature_selection import RFE
from sklearn.linear_model import LinearRegression
from statsmodels.stats.outliers_influence import variance_inflation_factor
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.preprocessing import PolynomialFeatures

from scipy import stats
from scipy.stats import norm, kurtosis, skew

# Import specific libraries
import statsmodels.api as sm
from statsmodels.stats import diagnostic as diag
from statsmodels.stats.outliers_influence import variance_inflation_factor



# Stats
from scipy.stats import skew, norm
from scipy.special import boxcox1p
from scipy.stats import boxcox_normmax

from scipy import stats
from scipy.stats import skew, norm, probplot, boxcox
from scipy.special import boxcox1p
from patsy import dmatrices

pd.set_option('expand_frame_repr', False)

In [ ]:
#Loading datasets
%store -r all_data
%store -r y_train
%store -r cols

In [ ]:
all_data.shape

In [ ]:
##El método de más arriba tira error de sintaxis con el feature 2ndFlrSF porque empieza con 2 (parece),así que rename

all_data.rename(columns={'2ndFlrSF':'SndFlrSF'}, inplace=True)

In [ ]:
cols

In [ ]:
###treatment of skewed features

#Seleccionamos los numeric features only (usamos cols porque esos son los feats que quedaron)
numeric_features = all_data.loc[:, list(cols)].dtypes[(all_data.dtypes != "category") & (all_data.dtypes != "uint8")].index

# #Calculamos los skewes y los ordenamos...
skewed_features = all_data[numeric_features].apply(lambda x: skew(x.dropna())).sort_values(ascending =False)

#Seleccionamos los que tengan un abs(skew) >0.7
skewed_max = pd.DataFrame({"Skew": skewed_features})
skewed_max  = skewed_max[abs(skewed_max["Skew"]) > 0.7]
skewed_max.dropna()

#Aplicamos ahora la BoxCox transformation
l_opt = {}
for f in skewed_max.index:
    all_data[f], l_opt[f] = boxcox((all_data[f] + 1))
    
#Checking the results...

skewed_features_2 = all_data[numeric_features].apply(lambda x: skew(x.dropna())).sort_values(ascending =False)
sk_2 = pd.DataFrame({"Skew_before":skewed_features, "Skew_after": skewed_features_2})
sk_2 = sk_2[abs(sk_2["Skew_before"]) > 0.7].sort_values(by = "Skew_before", ascending = False)
sk_2

In [ ]:
"""Casi todos los features quedaron con un skew aceptable y cercano al normal, aunque los dos primeros
todavía se ven flojos. Una análisis útil para ver cómo quedan y si aúns e pueden abordar con ajustes discretos
es el análisis gráfico: QQPlot y Density Plot"""

In [ ]:
##EVALUACION GRÁFICA DE LOS FEATURES TRANSFORMADOS

"""Método para plotear los features"""

def QQ_plot(data, feature_name):
    #Setting the canvas
    fig = plt.figure(figsize = (12,4))
    
    #Fit the parameters (estimates the parameters of the function)
    (mu, sigma) = norm.fit(data)
    
    #Plots the kernel density plot KD for the feature
    fig1 = fig.add_subplot(121) # una fila con 2 plots, empezando con el 1
    sns.distplot(data, fit = norm,  hist=True, kde_kws={'bw':0.05})
     

#     sns.distplot(data , fit = norm)
    fig1.set_title(feature_name + "Distribution (mu = {:.2f}  and sigma = {:.2f})".format(mu,sigma), loc = "center")
    fig1.set_xlabel(feature_name)
    fig1.set_ylabel("Frequency")
    
    #Plots de qq-plot
    fig2 = fig.add_subplot(122) # segundo plot
    res = probplot(data, plot = fig2)
    fig2.set_title(feature_name + " Probability plot (skewness: {:.6f} and kurtosis: {:.6f} )".format(data.skew(), data.kurt()),
                 loc = "center")
    plt.tight_layout()
    
    plt.show()
    
for f in skewed_max.index:
    QQ_plot(all_data[f], f)

In [ ]:
"""Puede verse que KitchenAbvGr y MiscVal siguen mostrando un alto skewness. Las otras variables muestran mejorías
importantes en sus skewness y en general también en sus kurtosis"""

In [ ]:
##ANALIZAR SI ES POSIBLE INCORPORAR POLYNOMIAL FEATURES WITH THE MOST CORRELATED ENIGINEERED FEATURES
"""Para analizar esta posibilidad primero revisamos cómo se comportan los features elegidos 
en relación con la variable target. Para ello ploteamos"""

##Most correlated ENGINEERED features:  ['ConstructArea', 'TotalPoints', 'LotAreaMultSlope',  'GarageArea_x_Car', "TotalExtraPoins"]. 


In [ ]:
pd.concat((all_data.iloc[:len(y_train), :], y_train), axis = 1).corr().SalePrice.sort_values(ascending = False)

In [ ]:
##Ploteamos ahora estos 5 features y vemos su comportamiento.Será lineal??

fig = plt.figure(figsize = (20,15))
ax  = fig.add_subplot(331)
g = sns.regplot(y = "SalePrice", x = "ConstructArea", data = pd.concat((all_data.iloc[:len(y_train), :], y_train), axis = 1), order = 3 )
ax  = fig.add_subplot(332)
g = sns.regplot(y = "SalePrice", x = "TotalExtraPoints", data = pd.concat((all_data.iloc[:len(y_train), :], y_train), axis = 1), order = 3)
ax  = fig.add_subplot(333)
g = sns.regplot(y = "SalePrice", x = "GarageArea_x_Car", data = pd.concat((all_data.iloc[:len(y_train), :], y_train), axis = 1), order = 3)
ax  = fig.add_subplot(334)
g = sns.regplot(y = "SalePrice", x = "LotAreaMultSlope", data = pd.concat((all_data.iloc[:len(y_train), :], y_train), axis = 1), order = 3)
ax  = fig.add_subplot(335)
g = sns.regplot(y = "SalePrice", x = "TotalPoints", data = pd.concat((all_data.iloc[:len(y_train), :], y_train), axis = 1), order = 3)


In [ ]:
"""Vemos que todos tiene una relación no-lineal con target, lo cual implica que una curva polinomial sería lo más
adecuado para describir la relación de la variable con el target (POLYNOMIAL REGRESSION), ya sea cuadrática, cúbica, etc. Esto implica entonces agregar
nuevos features que son polinomiales.
Más información: https://youtu.be/Qnt2vBRW8Io
https://machinelearningmastery.com/polynomial-features-transforms-for-machine-learning/

Diseñaremos y analizaremos nuevos polynomial features para elegir los que mejor resultado dan (R2). Se deja fuera TotalExtraPoints ya que contiene features que ya están en TotalPoints"""

In [ ]:
##METODO PARA EVALUAR LOS POLINOMIOS DE COMBINACIONES DE FEATURES

def poly(X, y, feat =""):
    """Recibe X como un feat o varios y genera polinomios, realiza regresiones y evalúa usando R2"""

    #Instanciamos para obtener los cudráticos, cubos, 4 y 5th 
    quadratic = PolynomialFeatures(degree = 2, interaction_only = False, include_bias= False)
    cubic = PolynomialFeatures(degree = 3, interaction_only = False, include_bias= False)
    fourth = PolynomialFeatures(degree = 4, interaction_only = False, include_bias= False)
    fifth = PolynomialFeatures(degree = 5, interaction_only = False, include_bias= False)

    #Creamos ahora los polinomios
    X_quad = quadratic.fit_transform(X)
    X_cubic= cubic.fit_transform(X)
    X_fourth= fourth.fit_transform(X)
    X_fifth = fifth.fit_transform(X)


    #We make the regression on each polynomial and compute R2

    #Linear fit
    regr = LinearRegression()
    regr = regr.fit(X,y)
    y_lin_predict = regr.predict(X)
    linear_r2 = r2_score(y, y_lin_predict)
    
    #Quadratic fit
    regr = regr.fit(X_quad,y)
    y_quad_predict = regr.predict(X_quad)
    quad_r2 = r2_score(y, y_quad_predict)
    
    #Cubic fit
    regr = regr.fit(X_cubic,y)
    y_cubic_predict = regr.predict(X_cubic)
    cubic_r2 = r2_score(y, y_cubic_predict)
    
     #Fourth fit
    regr = regr.fit(X_fourth,y)
    y_fourth_predict = regr.predict(X_fourth)
    fourth_r2 = r2_score(y, y_fourth_predict)
    
     #Fifth fit
    regr = regr.fit(X_fifth,y)
    y_fifth_predict = regr.predict(X_fifth)
    fifth_r2 = r2_score(y, y_fifth_predict)
    

    print("Results:")
    print("Linear:", linear_r2)
    print("Quadratic:", quad_r2)
    print("Cubic:",cubic_r2 )
    print("Fourth:", fourth_r2)
    print("Fifth:", fifth_r2)

    if len(feat)>0:
        fig = plt.figure(figsize=(20,5))
        # Plot lowest Polynomials
        fig1 = fig.add_subplot(121)
        plt.scatter(X[feat], y, label='training points', color='lightgray')
        plt.plot(X[feat], y_lin_predict, label='linear (d=1), $R^2=%.3f$' % linear_r2, color='blue', lw=0.5, linestyle=':')
        plt.plot(X[feat], y_quad_predict, label='quadratic (d=2), $R^2=%.3f$' % quad_r2, color='red', lw=0.5, linestyle='-')
        plt.plot(X[feat], y_cubic_predict, label='cubic (d=3), $R^2=%.3f$' % cubic_r2,  color='green', lw=0.5, linestyle='--')

        plt.xlabel(feat)
        plt.ylabel('Sale Price')
        plt.legend(loc='upper left')

        # Plot higest Polynomials
        fig2 = fig.add_subplot(122)
        plt.scatter(X[feat], y, label='training points', color='lightgray')
        plt.plot(X[feat], y_lin_predict, label='linear (d=1), $R^2=%.3f$' % linear_r2, color='blue', lw=2, linestyle=':')
        plt.plot(X[feat], y_fifth_predict, label='Fifth (d=5), $R^2=%.3f$' % fifth_r2, color='yellow', lw=2, linestyle='-')
        plt.plot(X[feat], y_fifth_predict, label='Fourth (d=4), $R^2=%.3f$' % fourth_r2, color='red', lw=2, linestyle=':')

        plt.xlabel(feat)
        plt.ylabel('Sale Price')
        plt.legend(loc='upper left')
    else:
        # Plot initialisation
        fig = plt.figure(figsize=(20,10))
        ax = fig.add_subplot(121, projection='3d')
        ax.scatter(X.iloc[:, 0], X.iloc[:, 1], y, s=40)

        # make lines of the regressors:
        plt.plot(X.iloc[:, 0], X.iloc[:, 1], y_lin_predict, label='linear (d=1), $R^2=%.3f$' % linear_r2, 
                 color='blue', lw=2, linestyle=':')
        plt.plot(X.iloc[:, 0], X.iloc[:, 1], y_quad_predict, label='quadratic (d=2), $R^2=%.3f$' % quad_r2, 
                 color='red', lw=0.5, linestyle='-')
        plt.plot(X.iloc[:, 0], X.iloc[:, 1], y_cubic_predict, label='cubic (d=3), $R^2=%.3f$' % cubic_r2, 
                 color='green', lw=0.5, linestyle='--')
        # label the axes
        ax.set_xlabel(X.columns[0])
        ax.set_ylabel(X.columns[1])
        ax.set_zlabel('Sales Price')
        ax.set_title("Poly up to 3 degree")
        plt.legend(loc='upper left')

    plt.tight_layout()
    plt.show()



In [ ]:
##Para facilitar el ploteo usamos dataset con saleprice
data = pd.concat((all_data.iloc[:len(y_train), :], y_train), axis = 1)
X = data.loc[:, ["ConstructArea"]]

poly(X, y_train, "ConstructArea" )

In [ ]:
X = data.loc[:, ['ConstructArea', 'TotalPoints']]
poly(X, y_train)

In [ ]:
X= data.loc[:, ['ConstructArea', 'TotalPoints', 'LotAreaMultSlope',  'GarageArea_x_Car']]
poly(X, y_train)

In [ ]:
##Se selecciona el último mix en tercer grado dado que da el mayor R2 (a pesar de que no es tan significativo lo que se gana respecto de
#la combinación lineal de los 4). Se selecciona el tercer grado debido a que la literatura indica que un mayor grado tiende a overfit

In [ ]:
##CREACION DE POLYNOMIAL FEATURES Y AGREGACIÓN AL DATASET

"""Generamos ahora los polynomial features en grado 3 con el mix de los 4 features seleccionados
y los agregamos al dataset"""

poly_cols = ['ConstructArea', 'TotalPoints', 'LotAreaMultSlope',  'GarageArea_x_Car']
pf = PolynomialFeatures(degree= 3, interaction_only= False, include_bias= False) #instaciamos
res= pf.fit_transform(all_data.loc[:, poly_cols]) #Se crearon 30 nuevos features a partir de los 4 del mix

#Ahora corresponde darles nombres y acoplarlos al dataset
names = pf.get_feature_names(poly_cols) #genera una lista con los nombres de los features creados

#Nos deshacemos de los espacios en blanco que aparecen en los nombres y los reemplazamos por _ 
target_feature_names = [f.replace(" ", "_") for f in names]

#Creo dataframe con los nuevos feats y sus nombres
new_poly_features = pd.DataFrame(data = res, columns= target_feature_names, index = all_data.index)

#Eliminamos los features madres para que no se repitan en el dataset

new_poly_features.drop(poly_cols, axis = 1, inplace = True)

#Acople final

all_data = pd.concat([all_data, new_poly_features], axis = 1)





In [ ]:
####NEW SHAPE OF DATASET AFTER POLYNOMIAL FEATURE CREATION
all_data.shape